In [ ]:
from huggingface_hub import login

# Paste your token here
login("***")

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("epfl-llm/meditron-7b")
model = AutoModelForCausalLM.from_pretrained("epfl-llm/meditron-7b")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.08k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.85M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

...

model-00005-of-00008.safetensors:   0%|          | 0.00/1.90G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.91G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:
!pip install deep_translator
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 124.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.0/73.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.3 MB/s eta 0:00:00


In [8]:
from deep_translator import GoogleTranslator  # For translation


In [9]:
from deep_translator import GoogleTranslator

SYSTEM_PROMPT = (
    "You are Meditron, a helpful and knowledgeable medical assistant. "
    "Always provide detailed, clear, and evidence-based responses to health-related questions. "
    "If the user asks about non-medical topics, answer politely but briefly."
)

def detect_language(text):
    """Very basic Turkish detection."""
    turkish_chars = "çğıöşüÇĞİÖŞÜ"
    return any(ch in turkish_chars for ch in text)

def chat_fn(message, history):
    history = history or []

    # Detect and translate to English if Turkish
    if detect_language(message):
        original_message = message
        message = GoogleTranslator(source='tr', target='en').translate(message)
    else:
        original_message = None

    # Build full conversation including system prompt
    conversation = f"System: {SYSTEM_PROMPT}\n"
    for user_msg, bot_msg in history:
        conversation += f"User: {user_msg}\nAssistant: {bot_msg}\n"
    conversation += f"User: {message}\nAssistant:"

    # Model inference
    inputs = tokenizer(conversation, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

    reply = tokenizer.decode(outputs[0], skip_special_tokens=True)
    reply = reply.split("Assistant:")[-1].strip()

    # Translate back to Turkish if original message was Turkish
    if original_message:
        reply = GoogleTranslator(source='en', target='tr').translate(reply)

    # Append original user message (not translated) to history
    history.append((original_message if original_message else message, reply))

    return reply


In [10]:
import gradio as gr
# Gradio interface
gr.ChatInterface(
    fn=chat_fn,
    title="🩺 Meditron Medical Chat",
    chatbot=gr.Chatbot(height=500),
    textbox=gr.Textbox(placeholder="Sorunuzu yazın..."),
).launch(share=True)

/tmp/ipython-input-1484870841.py:6: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=500),
/usr/local/lib/python3.11/dist-packages/gradio/chat_interface.py:328: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c9da404b1698af166e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
